<a href="https://colab.research.google.com/github/Karan345/DS_ML_Practice/blob/main/Torch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
a=torch.tensor(1.5)
b=torch.tensor(2.5)
c=torch.add(a,b)
print(c)

tensor(4.)


In [ ]:
print(torch.__version__)

2.0.1+cu118


### Building MLP

In [ ]:
import torch.nn as nn

model=nn.Sequential(
    nn.Linear(8,12),
    nn.ReLU(),
    nn.Linear(12,8),
    nn.ReLU(),
    nn.Linear(8,1),
    nn.Sigmoid()
)
print(model)

Sequential(
  (0): Linear(in_features=8, out_features=12, bias=True)
  (1): ReLU()
  (2): Linear(in_features=12, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=1, bias=True)
  (5): Sigmoid()
)


# Training a Pytorch Model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd

/content


In [9]:
import numpy as np
import torch.optim as optim

In [13]:
dataset =np.loadtxt('/content/drive/MyDrive/pima-indians-diabetes.csv', delimiter=',')


In [14]:
X = dataset[:,0:8]
y = dataset[:,8]
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)
 
loss_fn = nn.BCELoss() # binary cross-entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)
 
n_epochs = 100
batch_size = 10
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 0.553605318069458
Finished epoch 1, latest loss 0.5296656489372253
Finished epoch 2, latest loss 0.5039710998535156
Finished epoch 3, latest loss 0.4881969094276428
Finished epoch 4, latest loss 0.47227635979652405
Finished epoch 5, latest loss 0.46944618225097656
Finished epoch 6, latest loss 0.46543431282043457
Finished epoch 7, latest loss 0.4665314555168152
Finished epoch 8, latest loss 0.46502363681793213
Finished epoch 9, latest loss 0.46229100227355957
Finished epoch 10, latest loss 0.46324601769447327
Finished epoch 11, latest loss 0.46119385957717896
Finished epoch 12, latest loss 0.4536982774734497
Finished epoch 13, latest loss 0.45487555861473083
Finished epoch 14, latest loss 0.4519790709018707
Finished epoch 15, latest loss 0.45446011424064636
Finished epoch 16, latest loss 0.45784544944763184
Finished epoch 17, latest loss 0.45821574330329895
Finished epoch 18, latest loss 0.459485799074173
Finished epoch 19, latest loss 0.4618532061576843
F

In [15]:
i = 5
X_sample = X[i:i+1]
y_pred = model(X_sample)
print(f"{X_sample[0]} -> {y_pred[0]}")

tensor([  5.0000, 116.0000,  74.0000,   0.0000,   0.0000,  25.6000,   0.2010,
         30.0000]) -> tensor([0.2444], grad_fn=<SelectBackward0>)


In [16]:
i = 5
X_sample = X[i:i+1]
model.eval()
with torch.no_grad():
    y_pred = model(X_sample)
print(f"{X_sample[0]} -> {y_pred[0]}")

tensor([  5.0000, 116.0000,  74.0000,   0.0000,   0.0000,  25.6000,   0.2010,
         30.0000]) -> tensor([0.2444])


In [17]:
model.eval()
with torch.no_grad():
    y_pred = model(X)
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

Accuracy 0.7682291865348816
